## Student: Deng, Zixuan (V00971633)
# Problem 6


In [1]:
import pandas as pd
import numpy as np
import random
np.random.seed(1337)
random.seed(1337)

# Plotting support
from matplotlib import pyplot as plt
from plotnine import *
# Standard libraries
import pandas as pd
import sklearn as sk

## 1. [Regression; 20 points]
We will use the same setup as in Problem 2. This time, we will introduce
the following continuous features in addition to the existing categorical features: (normalized)
PerCapitaIncome and PovertyLevel. Your goal this time is to predict PovertyLevel given
other features (not the original label vector: that one should be ignored here). Again, provide
relevant metrics and plots as in Problem 2.

In [2]:
# to some reasons, I cannot read csv from the online link, so I upload the file. 
original_data = pd.read_csv('elections_clean.csv', header=0,index_col=0)
df = pd.read_csv('elections_clean.csv', header=0,index_col=0)


In [3]:
df.head()

,Votes,UnemploymentRate,HouseholdIncome,PerCapitaIncome,PovertyLevel,DeepPovertyLevel,Population,Area,PopDensity,MaleRate,FemaleRate,Turnout,Democrat,State,County,Education,Religion,Young,Adult,Old,EthnicMale,EthnicFemale
0,7471,0.032,0.687669,0.536632,0.162,0.049330,-0.303817,3.607604,-0.151766,0.4831,0.498326,0.022362,0,2,13,only high school diploma,Other Misc,0.25037,0.584595,0.096915,White Male,White Female
1,7471,0.038,1.756284,1.515032,0.099,0.042878,-0.296080,3.374070,-0.151244,0.4831,0.498326,0.013102,0,2,16,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female
2,7471,0.050,2.319570,2.182969,0.100,0.039032,0.619196,0.196708,-0.052741,0.4831,0.498326,0.000250,0,2,20,some college or associate's degree,Christian Generic,0.25037,0.584595,0.096915,White Male,White Female
3,7471,0.144,-0.110505,-0.909921,0.238,0.086448,-0.259464,11.578651,-0.151766,0.4831,0.498326,0.004163,0,2,50,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female
4,7471,0.092,2.333129,2.271372,0.095,0.048682,-0.310672,-0.083883,-0.150838,0.4831,0.498326,0.083756,0,2,60,some college or associate's degree,Catholic,0.25037,0.584595,0.096915,White Male,White Female


In [4]:
df.dtypes

Votes                 int64
UnemploymentRate    float64
HouseholdIncome     float64
PerCapitaIncome     float64
PovertyLevel        float64
DeepPovertyLevel    float64
Population          float64
Area                float64
PopDensity          float64
MaleRate            float64
FemaleRate          float64
Turnout             float64
Democrat              int64
State                 int64
County                int64
Education            object
Religion             object
Young               float64
Adult               float64
Old                 float64
EthnicMale           object
EthnicFemale         object
dtype: object

We need to convert categorical data to one-hot encoding. There are four features that are categorical: Education, Religion, EthnicMale, EthnicFemale. 

In [5]:
columns_demmy = ['Education','Religion','EthnicMale','EthnicFemale']
for i in columns_demmy:
    df = df.join(pd.get_dummies(df[i]))
    # print(pd.get_dummies(df[i]))

df.head()



,Votes,UnemploymentRate,HouseholdIncome,PerCapitaIncome,PovertyLevel,DeepPovertyLevel,Population,Area,PopDensity,MaleRate,FemaleRate,Turnout,Democrat,State,County,Education,Religion,Young,Adult,Old,EthnicMale,EthnicFemale,a bachelor's degree or higher,less than a high school diploma,only high school diploma,some college or associate's degree,Amish,Catholic,Christian Generic,Mormon,Other,Other Misc,Asian Male,White Male,Black Female,Native American Female,White Female
0,7471,0.032,0.687669,0.536632,0.162,0.049330,-0.303817,3.607604,-0.151766,0.4831,0.498326,0.022362,0,2,13,only high school diploma,Other Misc,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1
1,7471,0.038,1.756284,1.515032,0.099,0.042878,-0.296080,3.374070,-0.151244,0.4831,0.498326,0.013102,0,2,16,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
2,7471,0.050,2.319570,2.182969,0.100,0.039032,0.619196,0.196708,-0.052741,0.4831,0.498326,0.000250,0,2,20,some college or associate's degree,Christian Generic,0.25037,0.584595,0.096915,White Male,White Female,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
3,7471,0.144,-0.110505,-0.909921,0.238,0.086448,-0.259464,11.578651,-0.151766,0.4831,0.498326,0.004163,0,2,50,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
4,7471,0.092,2.333129,2.271372,0.095,0.048682,-0.310672,-0.083883,-0.150838,0.4831,0.498326,0.083756,0,2,60,some college or associate's degree,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1


In [6]:
df.head()

,Votes,UnemploymentRate,HouseholdIncome,PerCapitaIncome,PovertyLevel,DeepPovertyLevel,Population,Area,PopDensity,MaleRate,FemaleRate,Turnout,Democrat,State,County,Education,Religion,Young,Adult,Old,EthnicMale,EthnicFemale,a bachelor's degree or higher,less than a high school diploma,only high school diploma,some college or associate's degree,Amish,Catholic,Christian Generic,Mormon,Other,Other Misc,Asian Male,White Male,Black Female,Native American Female,White Female
0,7471,0.032,0.687669,0.536632,0.162,0.049330,-0.303817,3.607604,-0.151766,0.4831,0.498326,0.022362,0,2,13,only high school diploma,Other Misc,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1
1,7471,0.038,1.756284,1.515032,0.099,0.042878,-0.296080,3.374070,-0.151244,0.4831,0.498326,0.013102,0,2,16,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
2,7471,0.050,2.319570,2.182969,0.100,0.039032,0.619196,0.196708,-0.052741,0.4831,0.498326,0.000250,0,2,20,some college or associate's degree,Christian Generic,0.25037,0.584595,0.096915,White Male,White Female,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
3,7471,0.144,-0.110505,-0.909921,0.238,0.086448,-0.259464,11.578651,-0.151766,0.4831,0.498326,0.004163,0,2,50,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
4,7471,0.092,2.333129,2.271372,0.095,0.048682,-0.310672,-0.083883,-0.150838,0.4831,0.498326,0.083756,0,2,60,some college or associate's degree,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1


In [7]:
# drop the categorical feature
df = df.drop(['State', 'County',], axis=1)
df.head()

,Votes,UnemploymentRate,HouseholdIncome,PerCapitaIncome,PovertyLevel,DeepPovertyLevel,Population,Area,PopDensity,MaleRate,FemaleRate,Turnout,Democrat,Education,Religion,Young,Adult,Old,EthnicMale,EthnicFemale,a bachelor's degree or higher,less than a high school diploma,only high school diploma,some college or associate's degree,Amish,Catholic,Christian Generic,Mormon,Other,Other Misc,Asian Male,White Male,Black Female,Native American Female,White Female
0,7471,0.032,0.687669,0.536632,0.162,0.049330,-0.303817,3.607604,-0.151766,0.4831,0.498326,0.022362,0,only high school diploma,Other Misc,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1
1,7471,0.038,1.756284,1.515032,0.099,0.042878,-0.296080,3.374070,-0.151244,0.4831,0.498326,0.013102,0,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
2,7471,0.050,2.319570,2.182969,0.100,0.039032,0.619196,0.196708,-0.052741,0.4831,0.498326,0.000250,0,some college or associate's degree,Christian Generic,0.25037,0.584595,0.096915,White Male,White Female,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1
3,7471,0.144,-0.110505,-0.909921,0.238,0.086448,-0.259464,11.578651,-0.151766,0.4831,0.498326,0.004163,0,only high school diploma,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1
4,7471,0.092,2.333129,2.271372,0.095,0.048682,-0.310672,-0.083883,-0.150838,0.4831,0.498326,0.083756,0,some college or associate's degree,Catholic,0.25037,0.584595,0.096915,White Male,White Female,0,0,0,1,0,1,0,0,0,0,0,1,0,0,1


In [8]:
for i in range(20,35):
  df.iloc[:,i] = df.iloc[:,i].astype('int32')
df.iloc[:,20:35].dtypes

a bachelor's degree or higher         int32
less than a high school diploma       int32
only high school diploma              int32
some college or associate's degree    int32
Amish                                 int32
Catholic                              int32
Christian Generic                     int32
Mormon                                int32
Other                                 int32
Other Misc                            int32
Asian Male                            int32
White Male                            int32
Black Female                          int32
Native American Female                int32
White Female                          int32
dtype: object

In [9]:
df['PerCapitaIncome']

0       0.536632
1       1.515032
2       2.182969
3      -0.909921
4       2.271372
          ...   
3140    1.129145
3141    3.431841
3142    0.300889
3143    0.501551
3144    0.882001
Name: PerCapitaIncome, Length: 3145, dtype: float64

In [10]:
# undo the transformations done in Problem 1
def un_z_score_normalization(df):
   return df * df.std(ddof=0)+df.mean()

df['PerCapitaIncome'] = un_z_score_normalization(df['PerCapitaIncome'])
df['PovertyLevel'] = df['PovertyLevel'] * 100 

RMS is good. Just compare test/training errors, and compare tree depths and num of repeated features (if you can).

In [12]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

X = df.iloc[:,20:35]
X = X.join(df['PerCapitaIncome'])
y = df['PovertyLevel']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=6)
tree_clf = DecisionTreeRegressor(criterion = 'mse', splitter = 'best', max_depth = 5, max_leaf_nodes = 10, random_state = 6)
tree_clf.fit(X_train,y_train)
y_predict = tree_clf.predict(X_test)

print('Mean Squared Error:',metrics.mean_squared_error(y_test, y_predict))   




Mean Squared Error: 11.864592801538278


## 2. [Cross-validation; 20 points]
Roll out your own (implemented from scratch) 5-fold crossvalidation to select the best regression tree. You can use any decision tree implementation (your own, scikit-learn, etc.).

### K-fold Cross-validation
1. split data into k equal size sub datasets
2. For each sub dataset: 
  - each time, use one sub dataset as validation set, and rest sub datasets are for training purpose. 
  - for k = 5, each part is 20% of the data. 
3. Select the model with the best cross-validation error

In [13]:
# def k_fold_CV(data, k ): 
#   fold_size = int(data.shape[0] / k) 
#   data_split = []
  # for _ in range(k):
  #     fold = []
  #     while len(fold) < fold_size:
  #         # get random row index
  #         index = randrange(data.shape[0])
  #         fold.append(data.iloc[index,:])
  #         data.drop(index, inplace=True)
  #     data_split.append(np.asarray(fold))    
  # for i in range(k):
  #     validation = df.iloc[ i*fold_size : (i+1)* fold_size,:]
  #     training = pd.concat([df[:i*fold_size], df[(i+1)*fold_size :]])




  # return validation, training
      
# def kfoldCV(data, k):
#   data_split = datasplit(k, data)

#   for i in range(k): # repeat k times
#       # validation set 
#       validation_set = data_split[i]
#       if i!=k: 
#         training_set = 
#       for inde in range(k):
#         if inde != i:
#           training_set = pd.concat




In [14]:
# shuffle the data 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics



data = df.iloc[:,20:35]
data = data.join(df['PerCapitaIncome'])
data = data.join(df['PovertyLevel'])

data = shuffle(data)


fold_size = int(data.shape[0] / 5) 
data_split = []
mse = []

for i in range(5):
    validation = data.iloc[ i*fold_size : (i+1)* fold_size,:]
    training = pd.concat([data[:i*fold_size], data[(i+1)*fold_size :]])
    X_training = training.iloc[:,:16]
    X_validation = validation.iloc[:,:16]
    y_training = training['PovertyLevel']
    y_validation = validation['PovertyLevel']
    # print(X_training)
    tree_clf = DecisionTreeRegressor(criterion = 'mse', splitter = 'best', max_depth = 5, max_leaf_nodes = 10, random_state = 6)
    tree_clf.fit(X_training,y_training)
    y_predict = tree_clf.predict(X_validation)

    print('Mean Squared Error:',metrics.mean_squared_error(y_validation, y_predict))   
    mse.append(metrics.mean_squared_error(y_validation, y_predict))
print(mse)
average_mse = round(sum(mse)/5,2)
print("The average Mean Squre Error is",average_mse)
max_mse = round(max(mse),2)
min_mse = round(min(mse),2)
print("The Max Mean Squre Error is",max_mse)
print("The Min Mean Squre Error is",min_mse)


Mean Squared Error: 10.901210952013317
Mean Squared Error: 14.263157262822167
Mean Squared Error: 12.909485137236093
Mean Squared Error: 11.305141065269362
Mean Squared Error: 12.269573438636638
[10.901210952013317, 14.263157262822167, 12.909485137236093, 11.305141065269362, 12.269573438636638]
The average Mean Squre Error is 12.33
The Max Mean Squre Error is 14.26
The Min Mean Squre Error is 10.9


The best regerssion tree has the mean square error 10.9.